# XOR problem

## 1. Preliminaries
### 1.1. Imports
We begin by importing several python libraries:

In [1]:
%%bash
rm -rf train

In [2]:
import os
import tensorflow as tf
import numpy as np

i=0
logdir = os.getcwd()+'/train/log0'
while os.path.exists(logdir):
    i+=1
    logdir = os.getcwd()+'/train/log{}'.format(i)
os.makedirs(logdir)
    
print('tensorflow version: {}'.format(tf.__version__))
print('numpy version: {}'.format(np.__version__))
print('current working directory: {}'.format(os.getcwd()))
print('tensorboard logdir path: {}'.format(logdir))

tensorflow version: 1.1.0
numpy version: 1.12.1
current working directory: /Users/alexten/Projects/pdpyflow/xor
tensorboard logdir path: /Users/alexten/Projects/pdpyflow/xor/train/log0


### 1.2. Functions

In [3]:
def read_csv_file(filename_queue, batch_size, default_val, inp_size, targ_size, labels):
    reader = tf.TextLineReader(skip_header_lines=True, name='csv_reader')
    _, csv_row = reader.read_up_to(filename_queue, batch_size)
    defaults = [[default_val] for x in range(inp_size + targ_size)]
    if labels is True: 
        defaults.insert(0,[''])
    examples = tf.decode_csv(csv_row, record_defaults=defaults)
    l = tf.transpose(examples.pop(0))
    x = tf.transpose(tf.stack(examples[0:inp_size]))
    t = tf.transpose(tf.stack(examples[inp_size:inp_size + targ_size]))
    return l, x, t

### 1.3. Training environment and input pipeline

In [4]:
# CONFIGS
num_epochs = 330
batch_size = 4
inp_size = 2
targ_size = 1

# QUEUES
with tf.name_scope('Input_pipeline'):
    input_queue = tf.train.string_input_producer(
                    ['train_data_B.txt'], 
                    num_epochs = num_epochs, 
                    shuffle = False
    )

    labels, inp_batch, targ_batch = read_csv_file(
                                        filename_queue = input_queue,
                                        batch_size = batch_size,
                                        default_val = 0.0,
                                        inp_size = inp_size,
                                        targ_size = targ_size,
                                        labels = True
    )

## 2. Network construction

In [5]:
# CONFIGS
hidden_size = 2
wrange = [-1,1]
seed = None # Use None for random seed value
lr = 0.5
m = 0.9
use_exercise_params = True
ckpt_freq = 1
ecrit = 0.01

# NETWORK CONSTRUCTION
with tf.name_scope('XOR_model'):
    model_inp = tf.placeholder(dtype = tf.float32, shape=[batch_size, inp_size], name='model_inp')

    # HIDDEN LAYER
    with tf.name_scope('hidden_layer'):
        
        with tf.name_scope('weights'):
            hidden_W = tf.Variable(
                        tf.random_uniform(
                            minval = wrange[0], 
                            maxval = wrange[1],
                            seed = seed,
                            shape = [inp_size, hidden_size],
                            dtype=tf.float32
                        )
            )
            tf.summary.tensor_summary('tensor_summary', hidden_W)
                             
        with tf.name_scope('biases'):
            hidden_b = tf.Variable(
                        tf.random_uniform(
                            minval = wrange[0],
                            maxval = wrange[1],
                            seed = seed,
                            shape = [hidden_size],
                            dtype = tf.float32
                        )
            )
            tf.summary.tensor_summary('tensor_summary', hidden_b)

        with tf.name_scope('net_input'):
            hidden_net = tf.nn.xw_plus_b(model_inp, hidden_W, hidden_b)
            tf.summary.tensor_summary('tensor_summary', hidden_net)
        
        with tf.name_scope('activations'):
            hidden_acts = tf.nn.sigmoid(hidden_net)
            tf.summary.tensor_summary('tensor_summary', hidden_acts)
        
    # OUTPUT LAYER
    with tf.name_scope('output_layer'):
        
        with tf.name_scope('weights'):
            output_W = tf.Variable(
                        tf.random_uniform(
                            minval = wrange[0], 
                            maxval = wrange[1],
                            seed = seed,
                            shape = [hidden_size, targ_size],
                            dtype=tf.float32
                        )
            )
            tf.summary.tensor_summary('tensor_summary', output_W)
                          
        with tf.name_scope('biases'):
            output_b = tf.Variable(
                        tf.random_uniform(
                            minval = wrange[0],
                            maxval = wrange[1],
                            seed = seed,
                            shape = [targ_size],
                            dtype = tf.float32
                        )
            )
            tf.summary.tensor_summary('tensor_summary', output_b)
        
        with tf.name_scope('net_input'):
            output_net = tf.nn.xw_plus_b(hidden_acts, output_W, output_b)
            tf.summary.tensor_summary('tensor_summary', output_net)
        
        with tf.name_scope('activations'):
            output_acts = tf.nn.sigmoid(output_net, name='activations')
            tf.summary.tensor_summary('tensor_summary', output_acts)

with tf.name_scope('train'):
    target = tf.placeholder(dtype = tf.float32, shape=[batch_size, targ_size], name='model_inp')
    squared_error = tf.reduce_sum(tf.squared_difference(target, output_acts),
                                name='squared_error')
    tf.summary.scalar('error_summary', squared_error)
    train_step = tf.train.MomentumOptimizer(lr, m).minimize(squared_error)

merge_summaries = tf.summary.merge_all()

## 3. Running graph

In [6]:
sum_freq = 30 # (num_epochs // 10)
with tf.Session() as sess:
    summary_writer = tf.summary.FileWriter(logdir, sess.graph)
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    if use_exercise_params:
        restore_dict = {'w_1': hidden_W,'b_1': hidden_b,'w_2': output_W,'b_2': output_b}
        restore_op = tf.train.Saver(restore_dict)
        restore_op.restore(sess, 'exercise_params/exercise_params')
    coordinator = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coordinator)
    for i in range(num_epochs+1):
            try:
                x, t = sess.run([inp_batch, targ_batch])
                _, loss, summary = sess.run([train_step, squared_error, merge_summaries], feed_dict={model_inp: x, target: t})
                if i % sum_freq == 0 or i == num_epochs - 1:
                    summary_writer.add_summary(summary, i)
                    print('epoch {}: {}'.format(i,loss))
                if loss < ecrit:
                    summary_writer.add_summary(summary, i)
                    print('stop epoch {}: {}'.format(i,loss))
                    summary_writer.close()
                    break
            except tf.errors.OutOfRangeError:
                print('Reached the end of trainining set')
                break

INFO:tensorflow:Restoring parameters from exercise_params/exercise_params
epoch 0: 1.0506523847579956
epoch 30: 1.0019646883010864
epoch 60: 1.0000269412994385
epoch 90: 0.9999931454658508
epoch 120: 0.999982476234436
epoch 150: 0.9999575018882751
epoch 180: 0.9998661279678345
epoch 210: 0.9992285370826721
epoch 240: 0.9822084903717041
epoch 270: 0.6883569359779358
epoch 300: 0.043961603194475174
stop epoch 317: 0.009680739603936672


In [7]:
tf.reset_default_graph() # might be needed